In [1]:
import numpy
# scipy.special for the sigmoid function expit()
import scipy.special
# library for plotting arrays
import matplotlib.pyplot
# ensure the plots are inside this notebook, not an external window
%matplotlib inline


In [2]:
# neural network class definition
class neuralNetwork:
    
    
    # initialise the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))

        # learning rate
        self.lr = learningrate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass

    
    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = numpy.dot(self.who.T, output_errors) 
        
        # update the weights for the links between the hidden and output layers
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        pass

    
    # query the neural network
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

In [3]:
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

# learning rate
learning_rate = 0.1

# create instance of neural network
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

In [5]:


# load the mnist training data CSV file into a list
training_data_file = open("/home/sinch/Downloads/mnist_train_100.csv",'r')
training_data_list = training_data_file.readlines()
training_data_file.close()



In [25]:


# train the neural network

# epochs is the number of times the training data set is used for training
epochs = 10

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
    pass



In [26]:


# load the mnist test data CSV file into a list
test_data_file = open("/home/sinch/Downloads/mnist_test_10.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()




In [27]:


# test the neural network

# scorecard for how well the network performs, initially empty
scorecard = []

# go through all the records in the test data set
for record in test_data_list:
    # split the record by the ',' commas
    all_values = record.split(',')
    # correct answer is first value
    correct_label = int(all_values[0])
    # scale and shift the inputs
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    # query the network
    outputs = n.query(inputs)
    # the index of the highest value corresponds to the label
    label = numpy.argmax(outputs)
    # append correct or incorrect to list
    if (label == correct_label):
        # network's answer matches correct answer, add 1 to scorecard
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        pass
    
    pass



In [28]:


# calculate the performance score, the fraction of correct answers
scorecard_array = numpy.asarray(scorecard)
print ("performance = ", scorecard_array.sum() / scorecard_array.size)



performance =  0.7


In [10]:
#get first test record

all_values=test_data_list[0].split(',')
print(all_values[0])

7


In [11]:
n.query((numpy.asfarray(all_values[1:])/255.0*0.99)+0.01)

array([[0.06794855],
       [0.03998491],
       [0.06651486],
       [0.10050083],
       [0.10920573],
       [0.03947588],
       [0.01636532],
       [0.74910193],
       [0.10263951],
       [0.08122997]])

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,  51., 159., 253., 159.,  50.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0

In [7]:
#scale input to range 0.01 to 1.00

scaled_input = (numpy.asfarray(all_values[1:])/ 255.0 * 0.99) + 0.01
print(scaled_input)

[0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0

In [8]:
#output node is 0 for example

onodes = 10
targets = numpy.zeros(onodes) + 0.01
targets[int(all_values[0])]=0.99

In [9]:
print(targets)

[0.99 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01]


In [3]:
import numpy
import scipy.special #for sigmoid function expit()

#neural network class def
class neuralNetwork:

  #initialize the neural network
  def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
    #set no. of nodes in each input,hidden and output layer
    self.inodes=inputnodes
    self.hnodes=hiddennodes
    self.onodes=outputnodes



    #link weight matrices wih and who

    # weights inside the arrays are w_i_j, where link is from node
    #i to node j in the next layer
    # w11 w21
    # w12 w22 etc


   # self.wih = (numpy.random.rand(self.hnodes,self.onodes) - 0.5)
   # self.who = (numpy.random.rand(self.onodes,self.hnodes) - 0.5)

    self.wih = numpy.random.normal(0.0, pow(self.hnodes,0.5), (self.hnodes, self.inodes))
    self.who = numpy.random.normal(0.0, pow(self.onodes,0.5), (self.onodes, self.hnodes))



    
    #learning rate
    self.lr=learningrate


    #activation function is the sigmoid function

    self.activation_function = lambda x:scipy.special.expit(x)

    pass

  #train the neural network
  def train(self,inputs_list,targets_list):
    #convert inputs list into 2d array
    inputs=numpy.array(inputs_list,ndmin=2).T
    targets=numpy.array(targets_list,ndmin=2).T

    #calculate signals into hidden layer
    hidden_inputs=numpy.dot(self.wih,inputs)
    #calculate the signals emerging from hidden layer
    hidden_outputs=self.activation_function(hidden_inputs)

    #calculate signals into final layer
    final_inputs=numpy.dot(who,hidden_outputs)
    #calculate the signals emerginf from final layer
    final_outputs=self.activation_function(final_inputs)

    #o/p layer error is target-actual
    output_errors=targets-final_outputs

    #hidden layer error is output_error,split by weights,recombined at hidden nodes
    hidden_errors=numpy.dot(self.who.T,output_errors)


    #update weights for links between hidden and output layers
    self.who += self.lr * numpy.dot((output_errors * final_outputs * (1-final_outputs)),numpy.transpose(hidden_outputs))
    self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1-hidden_outputs)),numpy.transpose(inputs))



    pass

  #query the neural network
  def query(self,inputs_list):
    #convert input list into 2d array
    inputs=numpy.array(inputs_list,ndmin=2).T

    #calculate signals into hidden layer
    hidden_inputs=numpy.dot(self.wih,inputs)

    #calculate signals emerging from hidden layer
    hidden_outputs=self.activation_function(hidden_inputs)

    #calculate signals into final input layer
    final_inputs=numpy.dot(self.who,hidden_outputs)

    #calculate final outputs
    final_outputs=self.activation_function(final_inputs)

    return final_outputs
    pass

In [15]:
#no. of input,hidden and output nodes. This is code to create object
input_nodes = 784
hidden_nodes = 100
output_nodes = 10

#learning rate
learning_rate = 0.1

#create instance of neural network
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes,learning_rate)


# load the mnist training data CSV file into a list




ValueError: I/O operation on closed file.